# 2. 법령 문서를 벡터 DB에 저장하고, 질문에 맞는 문서를 찾아 LLM에게 전달해서 답변을 생성하는 기본 RAG 시스템 구축
1. 문서의 내용을 읽고 쪼갠다
2. 임베딩 -> 벡터 데이터베이스에 저장
3. 사용자 질문에 대한 벡터 데이터베이스에서의 유사도 검색 수행
4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
5. keyword 사전 활용하여 일상 용어를 법령 용어로 바꾸어 검색 정확도 높이기 

# 1. 문서의 내용을 읽고 쪼개기 - Knowledge Base 구성을 위한 데이터 생성
- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
  - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
  - 비용 감소와 답변 생성시간 감소의 효과
  - LangChain에서 다양한 TextSplitter들을 제공
- chunk_size 는 split 된 chunk의 최대 크기
- chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 한 조각 최대 1500자 
    chunk_overlap=200 # 앞뒤 200자씩 겹치게 함 (문맥이 끊기지 않도록)
)

loader = Docx2txtLoader('../../resouces/tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
document_list[52]

Document(metadata={'source': '../../resouces/tax_with_markdown.docx'}, page_content='⑧ 제1항ㆍ제4항 및 제5항에 따른 공제는 해당 거주자가 대통령령으로 정하는 바에 따라 신청한 경우에 적용하며, 공제액이 그 거주자의 해당 과세기간의 합산과세되는 종합소득금액을 초과하는 경우 그 초과하는 금액은 없는 것으로 한다.<개정 2012. 1. 1., 2014. 1. 1.>\n\n1. 삭제<2014. 1. 1.>\n\n2. 삭제<2014. 1. 1.>\n\n⑨ 삭제<2014. 1. 1.>\n\n⑩ 제1항ㆍ제4항ㆍ제5항 및 제8항에 따른 공제를 “특별소득공제”라 한다.<개정 2014. 1. 1.>\n\n⑪ 특별소득공제에 관하여 그 밖에 필요한 사항은 대통령령으로 정한다.<개정 2014. 1. 1.>\n\n[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제53조(생계를 같이 하는 부양가족의 범위와 그 판정시기) ① 제50조에 규정된 생계를 같이 하는 부양가족은 주민등록표의 동거가족으로서 해당 거주자의 주소 또는 거소에서 현실적으로 생계를 같이 하는 사람으로 한다. 다만, 직계비속ㆍ입양자의 경우에는 그러하지 아니하다.\n\n② 거주자 또는 동거가족(직계비속ㆍ입양자는 제외한다)이 취학ㆍ질병의 요양, 근무상 또는 사업상의 형편 등으로 본래의 주소 또는 거소에서 일시 퇴거한 경우에도 대통령령으로 정하는 사유에 해당할 때에는 제1항의 생계를 같이 하는 사람으로 본다.\n\n③ 거주자의 부양가족 중 거주자(그 배우자를 포함한다)의 직계존속이 주거 형편에 따라 별거하고 있는 경우에는 제1항에도 불구하고 제50조에서 규정하는 생계를 같이 하는 사람으로 본다.\n\n④ 제50조, 제51조 및 제59조의2에 따른 공제대상 배우자, 공제대상 부양가족, 공제대상 장애인 또는 공제대상 경로우대자에 해당하는지 여부의 판정은 해당 과세기간의 과세기간 종료일 현재의 상황에 따른다. 다만, 과세기

# 2. 임베딩 모델 생성 및 벡터 DB 저장
- 임베딩이란? 텍스트를 숫자 배열로 반환하는 것 (이를 통해 '의미 유사도' 계산이 가능함)
- Chroma를 통한 벡터 DB 저장

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv() # 파일에서 API KEY 불러오기 

embedding = UpstageEmbeddings(model='solar-embedding-1-large') # 임베딩 모델 생성 

In [4]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="../../chroma")

# 이미 저장된 데이터를 사용할 때 
#database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 3. 사용자 질문에 대한 벡터 데이터베이스에서의 유사도 검색 수행
- 질문 쿼리를 벡터화하여 Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [5]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)
retrieved_docs

[Document(id='b1c571ef-0bd8-4cca-992d-3f6694e02c0e', metadata={'source': '../../resouces/tax_with_markdown.docx'}, page_content='|-----------------------------------\t|-----------------------------------------------------\t|\n\n| 130만원 이하                      \t| 산출세액의 100분의 55                               \t|\n\n| 130만원 초과                      \t| 71만 5천원 + (130만원을 초과하는 금액의 100분의 30) \t|\n\n\n\n② 제1항에도 불구하고 공제세액이 다음 각 호의 구분에 따른 금액을 초과하는 경우에 그 초과하는 금액은 없는 것으로 한다.<신설 2014. 1. 1., 2015. 5. 13., 2022. 12. 31.>\n\n1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용
- RetrievalQA를 통해 LLM에 전달
  - RetrievalQA는 create_retrieval_chain으로 대체됨
  - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [6]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,                                # ChatUpstage
    retriever=database.as_retriever(),  # 검색기
    chain_type_kwargs={"prompt": prompt}
)

In [9]:
ai_message = qa_chain.invoke({"query": query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': "연봉 5천만원의 경우, 소득세는 71만 5천원입니다. 이는 130만원 초과 구간에 해당하며, 계산 방식은 '130만원을 초과하는 금액의 100분의 30'입니다. 5천만원 - 130만원 = 4천 870만원이며, 4천 870만원의 30%는 1천 461만원입니다. 따라서, 130만원 + 1천 461만원 = 1천 591만원이 산출세액이 됩니다. 그러나, 세액 공제 등 다른 요소를 고려하지 않았으므로, 실제 소득세는 다를 수 있습니다."}

# 5. keyword 사전 활용하여 일상 용어를 법령 용어로 바꾸어 검색 정확도 높이기 
- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 오직 질문만 그대로 리턴해주세요 다른 말은 절대 포함하지 마세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [11]:
new_question = dictionary_chain.invoke({"question": query})

In [12]:
new_question

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [13]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [14]:
ai_response = tax_chain.invoke({"question": query})
ai_response

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 390만원입니다. 이는 종합소득 과세표준 5천만원에 해당하는 세율 24%를 적용한 후, 기본공제 150만원을 차감하여 계산한 금액입니다.'}